# HandsOn2 - game
## Mahsa Massoud - 810098032

### Main Project:

In this project, we create an agent that plays blackjack with another agent in a deterministic environment. the algorithm used for finding agent's best decisions is MiniMax. We play with basic minimax algorithm and also with a method called Alpha-Beta Pruning to speed up search and prune search tree.

### Changes:

a boolean parameter which called prune is added tu the Blackjack class inorder to prevent duplication in code.


### How it works?

starting from current state as a Max node ,we try to maximize player's expected reward. we check all possible moves and with enlisting the help of heuristic function , we check whether any of player or component have won. then we pass game to next level of tree where is a Min level. in Min nodes we simulate reverse the roll of opoonent and player moves by trying to minimize game's result as -1 is win for opponent and 1 is win for player. this tree continues to grow to leafs where two players has stopped. each leaf is a final state of game , which is implemented in the check_for_winners methd,and then we backtrack to parent nodes.
for assigning suitable heuristic , we calculate the distance of player and opponent to the limit of 41. if any pass this bond, a coefficient which its absoulte value is high(100) will be multiplied to the difference between sum of the current cards and the target. eventually it will be added to the heuristic value and returned.  

### Q1 - 
No we can't use the min max directly but we can use a modified version which is Expectimax and use the probability of outcomes and use expected values to determine which move should we take.

### Q2,3 - 

in this game with some observations on different strategies, I find out that in each state starting with erasing cards from agents deck or opponent's deck is better than drawing card from game's deck and also drawing card is more better than stopping. the intuition behind this strategy is that stopping too early can give your opponent chance to remove card from your deck and at the same time opponent can draw card so it results in increase of opponents score and decreasing your score. also drawing cards can be dangerous as number of your remaining erases decreases and we get closer to end of game, drawing cards has risk of surpasing the limited 41 score.

In [32]:
import random
import copy
depth =3
import math
moves = ['d', 's', 'es', 'eo']

In [4]:
class Player:
    def __init__(self, name, num_of_cards):
        """
        The base player class of the game
        Inputs
        -----------
        name = (str) player's name
        num_of_cards = (int) number of cards in the deck
        """
        self.name = name
        self.deck_count = num_of_cards
        self.target = self.deck_count * 2 - 1
        self.cards = []
        self.erases_remaining = self.deck_count // 5
        self.has_stopped = False

    def draw_card(self, card):
        """
        draws a card, and adds it to player cards
        Input
        -------------
        card: (int) the card to be added
        """
        self.cards.append(card)

    def print_info(self):
        """
        prints info of the player
        """
        print(f"{self.name}'s cards: ", end='')
        for c in self.cards:
            print(f'{c}, ', end='')
        print(f'sum: {sum(self.cards)}')
    
    def get_margin(self):
        """
        returns the margin left to target by the player
        Output
        ----------
        (int) margin to target
        """
        return self.target - sum(self.cards)
    
    def cpu_play(self, seen_cards, deck, enemies_cards):
        """
        The function for cpu to play the game
        Inputs
        ----------
        seen_cards:     (list of ints) the cards that have been seen until now
        deck:           (list of ints) the remaining playing deck of the game
        enemies_cards:  (list of ints) the cards that the enemy currently has.
        Output
        ----------
        (str) a command given to the game
        
        """
        if (len(deck) > 0):
            next_card_in_deck = deck[0]
        else:
            next_card_in_deck = 0
        if (len(deck) > 1):
            next_enemy_card_in_deck = deck[1]
        else:
            next_enemy_card_in_deck = 0
        amount_to_target = self.target - sum(self.cards)
        amount_with_next_card = self.target - (sum(self.cards) + next_card_in_deck)
        enemies_amount_to_target = self.target - sum(enemies_cards)
        enemies_amount_with_next_card = self.target - (sum(enemies_cards) + next_enemy_card_in_deck)
        _stop_condition = amount_to_target < next_card_in_deck and self.erases_remaining <= 0
        _draw_condition_1 = next_card_in_deck != 0
        _draw_condition_2 = amount_with_next_card >= 0
        _erase_condition = self.erases_remaining > 0
        _erase_self_condition = amount_to_target < 0
        _erase_opponent_condition_or = enemies_amount_to_target < (self.target // 7)
        _erase_opponent_condition_or_2 = enemies_amount_with_next_card < (self.target // 7) 
        _erase_opponent_condition_or_3 = enemies_amount_with_next_card <= amount_with_next_card
        _erase_opponent_condition_or_4 = enemies_amount_to_target <= amount_to_target
        _erase_opponent_condition = _erase_opponent_condition_or or _erase_opponent_condition_or_2 or _erase_opponent_condition_or_3
        _erase_opponent_condition = _erase_opponent_condition or _erase_opponent_condition_or_4 
        if (_stop_condition):
            return 'stop'
        elif (_draw_condition_1 and _draw_condition_2):
            return 'draw'
        elif(_erase_self_condition and _erase_condition):
            return 'erase_self'
        elif(_erase_opponent_condition and _erase_condition):
            return 'erase_opponent'
        else:
            return 'stop'
    
    def erase(self, target):
        """
        erases the last card of the target
        Input
        ---------
        target: (Player obj) the player whos last card is about to be erased
        """
        if (len(target.cards) == 0):
            print(f'{target.name} has no more eraseble cards!')
            return
        if (self.erases_remaining > 0):
            self.erases_remaining -= 1
            card = target.cards.pop(-1)
            print(f'{self.name} erased {card} from {target.name}\'s deck!')
            return
        print(f'{self.name} has no more erases remaining!')

    def get_player_cards(self):
        return self.cards

    def get_erases_remained(self):
        return self.erases_remaining

In [31]:
class Blacksin:
    def __init__(self, prune,deck_count=21):
        """
        The main game class
        Inputs
        -----------
        deck_count = (int) number of cards in the deck
        """
        self.deck_count = deck_count
        self.target = self.deck_count * 2 - 1
        self.player = Player('player', deck_count)
        self.opponent = Player('opponent', deck_count)
        self.deck = self.shuffle_cards()
        self.seen_cards = []
        self.prune = prune
    
    def shuffle_cards(self):
        """ 
        shuffles cards for deck creation
        """
        return list(random.sample(range(1, self.deck_count + 1), self.deck_count))

    def draw_card(self):
        """ 
        draws a card from deck, if non is remaining, ends the game.
        """
        if (len(self.deck) > 0):
            card = self.deck.pop(0)
            self.seen_cards.append(card)
            return card
        print('The deck is empty! ending game...')
        self.opponent.has_stopped = True
        self.player.has_stopped = True
        return -1

    def handout_cards(self):
        """ 
        hands out cards to players
        """
        self.player.draw_card(self.draw_card())
        self.opponent.draw_card(self.draw_card())
        self.player.draw_card(self.draw_card())
        self.opponent.draw_card(self.draw_card())
    
    def handle_input(self, _input, player):
        """ 
        handles input
        Input
        ------------
        _input: (str) input given by the player
        player: (Player obj)the player that is giving the input
        
        """
        if (player is self.player):
            opponent = self.opponent
        else:
            opponent = self.player
        if (_input == 'stop' or _input == 's'):
            player.has_stopped = True
            print(f'{player.name} has stopped')
        elif (_input == 'draw' or _input == 'd'):
            card = self.draw_card()
            if (card == -1): return True
            player.draw_card(card)
            print(f'{player.name} drawed a card: {card}')
        elif ((_input == 'erase_self' or _input == 'es')):
            player.erase(player)
        elif ((_input == 'erase_opponent' or _input == 'eo')):
            player.erase(opponent)
        else:
            print('ERROR: unknown command')
            return False
        return True

    def get_player_input(self):
        """TODO"""
        res = self.minimax(state=True, max_depth = 1)
        your_input = res[1]
        self.handle_input(your_input, self.player)
            
    def opponent_play(self):
        """
        function for opponent to play it's turn
        """
        try:
            opponent_input = self.opponent.cpu_play(self.seen_cards, self.deck, self.player.cards)
        except:
            opponent_input = 'stop'
        self.handle_input(opponent_input, self.opponent)

    def check_for_winners(self):
        """
        checks for winners.
        Output
        -----------
        (int) returns 1 if player wins, 0 if draw and -1 if opponent wins
        """
        self.opponent.print_info()
        self.player.print_info()
        player_margin = self.player.get_margin()
        opponent_margin = self.opponent.get_margin()
        player_win_condition_1 = opponent_margin < 0 and player_margin >= 0
        player_win_condition_2 = opponent_margin >=0 and player_margin >= 0 and player_margin < opponent_margin
        draw_condition_1 = opponent_margin < 0 and player_margin < 0
        draw_condition_2 = opponent_margin >= 0 and player_margin >= 0 and player_margin == opponent_margin
        opponent_win_condition_1 = player_margin < 0 and opponent_margin >= 0
        opponent_win_condition_2 = opponent_margin >=0 and player_margin >= 0 and player_margin > opponent_margin
        if (player_win_condition_1 or player_win_condition_2):
            print(f'the winner is the {self.player.name}!')
            return 1
        elif(draw_condition_1 or draw_condition_2):
            print('the game ends in a draw!')
            return 0
        elif(opponent_win_condition_1 or opponent_win_condition_2):
            print(f'the winner is the {self.opponent.name}!')
            return -1
        else:
            print('an error has accurred! exiting...')
            exit()

    def print_deck(self):
        """
        prints the current deck of the game
        """
        print('full deck: [top] ', end='')
        for i in self.deck:
            print(i, end=' ')
        print('[bottom]')

    def run(self):
        """
        main function to run the game with
        """
        print('\nstarting game... shuffling... handing out cards...')
        print(f'remember, you are aiming for nearest to: {self.target}')
        self.print_deck()
        self.handout_cards()
        turn = 0
        while(not self.player.has_stopped or not self.opponent.has_stopped):
            if (turn == 0):
                if (not self.player.has_stopped):
                    self.opponent.print_info()
                    self.player.print_info()
                    self.get_player_input()
                    print()
            else:
                if (not self.opponent.has_stopped):
                    print('opponent playing...')
                    self.opponent_play()
                    print()
            turn = 1 - turn
        print('\nand the winner is...')
        return self.check_for_winners()
    
    def heuristic(self, a):
        self_sum = sum(self.player.cards[:])
        opponent_sum = sum(self.opponent.cards[:])
        h = self_sum - opponent_sum 

        if(self_sum > self.target):
            h -= a * (self_sum - self.target)
        if(opponent_sum > self.target):
            h += a * (opponent_sum - self.target)

        return h

    def minimax(self, state, max_depth , a = 0 , b = 0):
    
        if (self.player.has_stopped  and self.opponent.has_stopped ) or (max_depth > depth):
            return (self.heuristic(100) , '') 

        if (state):
            player = self.player
            opponent = self.opponent
        else:
            player = self.opponent
            opponent = self.player

        result = [] 

        if (player.has_stopped and not opponent.has_stopped):
            child = copy.deepcopy(self)
            answer = child.minimax(not(state) , max_depth+1, a , b)
            return (answer[0], '')


        for i in range(4):
            Min =  math.inf
            Max =  -math.inf
            child = copy.deepcopy(self)
            if(i == 0 and len(self.deck) < 1 ):
                continue
            if(i==1 and player.has_stopped == True):
                continue 
            if(i==2 and not ((player.get_erases_remained() > 0) and (len(player.get_player_cards()) > 0)) ):
                continue
            if(i == 3 and not ((player.erases_remaining > 0) and (len(opponent.get_player_cards()) > 0)) ):
                continue

            if(state):
                cur_player = child.player
            else:
                cur_player = child.opponent
            child.handle_input(moves[i], cur_player)

            answer = child.minimax(not(state), max_depth+1, a , b)

            if (self.prune):
                if (state):
                    Max = max(Max, answer[0])
                    if (Max >= b):
                        return ((answer[0], moves[i]))
                    a = max(a, Max)
                else:
                    Min = min(Min, answer[0])
                    if (Min <= a):
                        return ((answer[0], moves[i]))
                    b = min(b, Min)

            result.append((answer[0], moves[i]))


        result.sort(key = lambda t: t[0], reverse = state)

        return result[0]

In [34]:
# Example of using the game class
import time
depth = 3
win = 0
tic = time.time()
for i in range(500):
    game = Blacksin(prune = True , deck_count=21)
    result = game.run()
    if(result == 1):
        win+=1
toc = time.time()

print ("Time:", toc - tic) 
print("-------------------------------------------------------------------------")
print("win rate of the player =" , win / 5 )

In [35]:
toc- tic

32.36200022697449

### Results with pruning 

| depth | 3 | 4 | 6 |
| --- | --- | --- | --- |
|  win rate| 61.8% | 36% | 54.4% |
|  time| 12s| 17s|38s |


### Results without pruning 

| depth | 3 | 4 | 6 |
| --- | --- | --- | --- |
|win rate  | 80.6% | 64.2% | 58.4% |
| time | 34s| 104s|272s |

